In [1]:
# Import the dependencies
import pandas as pd
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Set the column width
pd.options.display.max_colwidth = 200

# Read the dataset and store as pandas dataframe. 
stations = pd.read_excel("Data/TTC Subway Stations.xlsx")
stations.head()

,ID,Station,Category,Address,CITY,Postal Code,Line,Grade,Ridership (2019[9] avg. weekday),Parking spaces[10],...,LAT min,LAT seconds,LAT direction,LAT decimal,LON,LON degree,LON min,LON seconds,LON direction,LON decimal
0,1,Finch,TTC Subway Station,5600 Yonge Street,North York,M2N 5S2,1,Underground,102025,"3,227[13]",...,46,50.0,N,43.780556,79°24?53?W,79,24,53.0,W,79.414722
1,2,North York Centre,TTC Subway Station,5102 Yonge Street,North York,M2N 6L8,1,Underground,31471,0,...,46,6.0,N,43.768333,79°24?46?W,79,24,46.0,W,79.412778
2,3,Sheppard–Yonge,TTC Subway Station,20 Sheppard Avenue West,North York,M2N 5M9,"1,4",Underground,134076,0,...,45,41.0,N,43.761389,79°24?39?W,79,24,39.0,W,79.410833
3,4,York Mills,TTC Subway Station,4015 Yonge Street,North York,M5M 3G1,1,Underground,28461,0,...,44,39.0,N,43.744167,79°24?24?W,79,24,24.0,W,79.406667
4,5,Lawrence,TTC Subway Station,3101 Yonge Street,Toronto,M4N 3N1,1,Underground,28041,0,...,43,30.0,N,43.725000,79°24?8?W,79,24,8.0,W,79.402222


In [3]:
# List out column names and data types to identify any changes that are needed
stations.dtypes

ID                                    int64
Station                              object
Category                             object
Address                              object
CITY                                 object
Postal Code                          object
Line                                 object
Grade                                object
Ridership (2019[9] avg. weekday)      int64
Parking spaces[10]                   object
Accessible[12]                       object
LAT                                  object
LAT degree                            int64
LAT min                               int64
LAT seconds                         float64
LAT direction                        object
LAT decimal                         float64
LON                                  object
LON degree                            int64
LON min                               int64
LON seconds                         float64
LON direction                        object
LON decimal                     

In [4]:
# Rename columns
stations.rename(columns={'CITY': "City",
                            'Ridership (2019[9] avg. weekday)': "Ridership_Weekly_Average",
                            'Parking spaces[10]': "Parking_Spaces",
                            'Accessible[12]': "Accessibility",
                            'Postal Code': 'Postal_code',
                            'LAT': 'LAT coordinate',
                            'LON': 'LON coordinate',
                            'LAT decimal': "LAT",
                            'LON decimal': "LON"}, inplace=True)

# Update station name to include 'TTC Subway Station' so 'Category' column can be removed
stations['Station'] = stations['Station'] + ' TTC Subway Station'

# Create new column to include province
stations['Province'] = 'Ontario'

In [5]:
# Trim the reference info from the parking space count
stations['Parking_Spaces'] = stations['Parking_Spaces'].str[:-4]

# Remove comma from the parking space count to prepare for integer conversion
# Fill in the NAN columns with 0 values instead
# Convert parking space count to integer
stations['Parking_Spaces'] = stations['Parking_Spaces'].str.replace(",","").fillna(0).astype(int)


In [6]:
# Confrm columns have been cleaned and identify the necessary ones only for export to mongodb
stations.dtypes

ID                            int64
Station                      object
Category                     object
Address                      object
City                         object
Postal_code                  object
Line                         object
Grade                        object
Ridership_Weekly_Average      int64
Parking_Spaces                int32
Accessibility                object
LAT coordinate               object
LAT degree                    int64
LAT min                       int64
LAT seconds                 float64
LAT direction                object
LAT                         float64
LON coordinate               object
LON degree                    int64
LON min                       int64
LON seconds                 float64
LON direction                object
LON                         float64
Province                     object
dtype: object

In [7]:
# Update dataframe to include only the necessary columns for the MongoDB export
stations = stations[['Station','Address', 'City', 'Province', 'Postal_code','Line','Ridership_Weekly_Average', 'Parking_Spaces','Accessibility', 'LAT', 'LON']]

# Print to confirm data has been cleaned
stations.head()

,Station,Address,City,Province,Postal_code,Line,Ridership_Weekly_Average,Parking_Spaces,Accessibility,LAT,LON
0,Finch TTC Subway Station,5600 Yonge Street,North York,Ontario,M2N 5S2,1,102025,3227,Yes,43.780556,79.414722
1,North York Centre TTC Subway Station,5102 Yonge Street,North York,Ontario,M2N 6L8,1,31471,0,Yes,43.768333,79.412778
2,Sheppard–Yonge TTC Subway Station,20 Sheppard Avenue West,North York,Ontario,M2N 5M9,"1,4",134076,0,Yes,43.761389,79.410833
3,York Mills TTC Subway Station,4015 Yonge Street,North York,Ontario,M5M 3G1,1,28461,0,Yes,43.744167,79.406667
4,Lawrence TTC Subway Station,3101 Yonge Street,Toronto,Ontario,M4N 3N1,1,28041,0,No,43.725000,79.402222


In [8]:
# Connect to MongoDB client and list database names to identify the database to be exported to
mongo = MongoClient(port=27017)
print(mongo.list_database_names())

['admin', 'ajunjee_db', 'autosaurus_db', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'mydb', 'petsitly_marketing', 'uk_food']


In [9]:
# Assign database to a variable
db = mongo['property_information_db']
# List out the collections in the database
print(db.list_collection_names())

[]


In [10]:
# Assign the collection to a variable
ttc_subway = db['ttc_subway_stations']

In [11]:
# Convert the dataframe to dictionary values and insert all entries into the specified collection
stations.reset_index(inplace=True)
data_dict = stations.to_dict("records")
ttc_subway.insert_many(data_dict)

InsertManyResult([ObjectId('662127ecac6cae29ef47b4b8'), ObjectId('662127ecac6cae29ef47b4b9'), ObjectId('662127ecac6cae29ef47b4ba'), ObjectId('662127ecac6cae29ef47b4bb'), ObjectId('662127ecac6cae29ef47b4bc'), ObjectId('662127ecac6cae29ef47b4bd'), ObjectId('662127ecac6cae29ef47b4be'), ObjectId('662127ecac6cae29ef47b4bf'), ObjectId('662127ecac6cae29ef47b4c0'), ObjectId('662127ecac6cae29ef47b4c1'), ObjectId('662127ecac6cae29ef47b4c2'), ObjectId('662127ecac6cae29ef47b4c3'), ObjectId('662127ecac6cae29ef47b4c4'), ObjectId('662127ecac6cae29ef47b4c5'), ObjectId('662127ecac6cae29ef47b4c6'), ObjectId('662127ecac6cae29ef47b4c7'), ObjectId('662127ecac6cae29ef47b4c8'), ObjectId('662127ecac6cae29ef47b4c9'), ObjectId('662127ecac6cae29ef47b4ca'), ObjectId('662127ecac6cae29ef47b4cb'), ObjectId('662127ecac6cae29ef47b4cc'), ObjectId('662127ecac6cae29ef47b4cd'), ObjectId('662127ecac6cae29ef47b4ce'), ObjectId('662127ecac6cae29ef47b4cf'), ObjectId('662127ecac6cae29ef47b4d0'), ObjectId('662127ecac6cae29ef47b4